In [2]:
import pandas as pd

In [3]:
# groupby(["分组字段1", "分组字段2", "分组字段3"])["聚合字段1", "聚合字段2", "聚合字段3"].聚合函数()

mango_series = pd.Series([4,5,6,1,1])
apple_series = pd.Series([5,4,3,0,2])
banana_series = pd.Series([2,3,5,2,7])

df = pd.DataFrame({"mango": mango_series, "apple": apple_series, "banana": banana_series})
df.groupby(["mango"])[["apple", "banana"]].mean()

# 这里可以使用groups查看分组结果，结果记录的是index而不是value\
print(df.groupby(["mango"]).groups)

# 使用get_group()来获取对应的分组信息，以分组字段的值作为key查询
print(df.groupby(["mango"]).get_group((4,)))
print("--------------")
for id, group in df.groupby(["mango"]):
    print(id, group)

{1: [3, 4], 4: [0], 5: [1], 6: [2]}
   mango  apple  banana
0      4      5       2
--------------
(1,)    mango  apple  banana
3      1      0       2
4      1      2       7
(4,)    mango  apple  banana
0      4      5       2
(5,)    mango  apple  banana
1      5      4       3
(6,)    mango  apple  banana
2      6      3       5


In [4]:
# concat

s1 = pd.Series(["A", "B"], index=[1,2])
s2 = pd.Series(["C", "D"], index=[3,4])
s3 = pd.Series(["E", "F"], index=[5,6])

pd.concat([s1, s2, s3], axis=0)
pd.concat([s1, s2, s3], axis=1)


,0,1,2
1,A,NaN,NaN
2,B,NaN,NaN
3,NaN,C,NaN
4,NaN,D,NaN
5,NaN,NaN,E
6,NaN,NaN,F


In [5]:
# merge
# how='inner' —— 内连接（intersection 交集）
# how='outer' —— 外连接（union 并集）
# how='left' —— 左连接（LEFT JOIN），保留左表（df1）中的所有 key，右表（df2）有匹配则填上，没有匹配的填 NaN。
# how='right' —— 右连接（RIGHT JOIN），保留右表（df2）中的所有 key，左表（df1）有匹配则填上，没有则 NaN。

# 一对一
df1 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen"], "grade": [99, 98, 10, 65]})
df2 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen"], "sex": ["M", "M", "F", "M"]})
df3 = pd.merge(df1, df2, how="inner", on="name")
print(df3)

# 一对多
df4 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen"], "location": ["US", "CN", "US", "JP"]})
df5 = pd.DataFrame({"location": ["US", "CN", "JP"], "no": ["1", "2", "3"]})
df6 = pd.merge(df4, df5, how="inner", on="location")
print(df6)

# 多对多
df7 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen"], "location": ["US", "CN", "US", "JP"]})
df8 = pd.DataFrame({"location": ["US", "US", "CN", "CN", "JP"], "skill": ["1", "2", "3", "4", "5"]})
df9 = pd.merge(df6, df8, how="inner", on="location")
print(df9)


      name  grade sex
0     kobe     99   M
1    kejun     98   M
2  lansrot     10   F
3     chen     65   M
      name location no
0     kobe       US  1
1    kejun       CN  2
2  lansrot       US  1
3     chen       JP  3
      name location no skill
0     kobe       US  1     1
1     kobe       US  1     2
2    kejun       CN  2     3
3    kejun       CN  2     4
4  lansrot       US  1     1
5  lansrot       US  1     2
6     chen       JP  3     5


In [6]:
# 两个DataFrame的key不一样，用left_on和right_on
# 一对一
df10 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen"], "grade": [99, 98, 10, 65]})
df11 = pd.DataFrame({"employee": ["kobe", "kejun", "lansrot", "chen"], "sex": ["M", "M", "F", "M"]})
df12 = pd.merge(df10, df11, how="inner", left_on="name", right_on="employee")
print(df12)


      name  grade employee sex
0     kobe     99     kobe   M
1    kejun     98    kejun   M
2  lansrot     10  lansrot   F
3     chen     65     chen   M


In [7]:
# 使用join来合并
# 有个前提条件，就是两个dataframe中不能有重复的列
# 使用lsuffix和rsuffix来使列名唯一
df13 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen"], "grade": [99, 98, 10, 65]})
df14 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen"], "sex": ["M", "M", "F", "M"]})
df13.join(df14, lsuffix="_left", rsuffix="_right")

,name_left,grade,name_right,sex
0,kobe,99,kobe,M
1,kejun,98,kejun,M
2,lansrot,10,lansrot,F
3,chen,65,chen,M


In [29]:
# 分组聚合，aggregate
df15 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen", "liu"], "grade": [99, 98, 10, 65, 100], "dept": ["ECE", "CS", "ECE", "ECE", "CS"]})
df15.groupby("dept")["grade"].max()
df15.groupby("dept")["grade"].min()
df15.groupby("dept")["grade"].median()

# 如何同时求这三个函数的
df15.groupby("dept")["grade"].agg(["min", "max", "median"])

# 可以向agg中传入字典，用来指示对那一列(行)做具体什么样的操作
df15.groupby("dept").agg({"grade":"sum"})

# 使用rename重命名
df15.groupby("dept").agg({"grade": "sum"}).rename(columns={"grade": "sum_grade"})

# 使用自定义函数来统计首字母
def first_func(s):
    res = set()
    for item in s:
        res.add(item[0])
    return res

df15.groupby("dept")["name"].agg(first_func)


dept
CS        {k, l}
ECE    {k, l, c}
Name: name, dtype: object

In [30]:
# 分组转换， transform
df16 = pd.DataFrame({"name": ["kobe", "kejun", "lansrot", "chen", "liu"], "grade": [99, 98, 10, 65, 100], "dept": ["ECE", "CS", "ECE", "ECE", "CS"]})

df15.groupby("dept")["grade"].transform("mean")


0    58.0
1    99.0
2    58.0
3    58.0
4    99.0
Name: grade, dtype: float64